In [2]:
%load_ext autoreload
%autoreload 2

import concurrent.futures
from cmipper import utils, config, parallelised_download_and_process, file_ops


In [2]:
# MVP, from https://stackoverflow.com/questions/44989473/nesting-concurrent-futures-threadpoolexecutor

def inner(i, j):
    return i, j, i**j


def outer(i):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(inner, i, j): j for j in range(5)}
        results = []
        for future in concurrent.futures.as_completed(futures):
            results.append(future.result())
    return results


def main():
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(outer, i): i for i in range(10)}
        results = []
        for future in concurrent.futures.as_completed(futures):
            results.extend(future.result())
    print(results)


if __name__ == "__main__":
    main()

[(3, 0, 1), (3, 1, 3), (3, 2, 9), (3, 3, 27), (3, 4, 81), (4, 0, 1), (4, 1, 4), (4, 2, 16), (4, 3, 64), (4, 4, 256), (5, 2, 25), (5, 0, 1), (5, 1, 5), (5, 3, 125), (5, 4, 625), (2, 0, 1), (2, 2, 4), (2, 1, 2), (2, 3, 8), (2, 4, 16), (6, 0, 1), (6, 2, 36), (6, 1, 6), (6, 3, 216), (6, 4, 1296), (0, 1, 0), (0, 0, 1), (0, 2, 0), (0, 3, 0), (0, 4, 0), (8, 1, 8), (8, 2, 64), (8, 0, 1), (8, 3, 512), (8, 4, 4096), (7, 2, 49), (7, 0, 1), (7, 1, 7), (7, 3, 343), (7, 4, 2401), (1, 2, 1), (1, 0, 1), (1, 3, 1), (1, 1, 1), (1, 4, 1), (9, 1, 9), (9, 0, 1), (9, 2, 81), (9, 3, 729), (9, 4, 6561)]


In [9]:
model_info_dict = utils.read_yaml(config.model_info)
download_config_dict = utils.read_yaml(config.download_config)
utils.limit_model_info_dict(model_info_dict, download_config_dict)['EC-Earth3P-HR']["variable_dict"].keys()


    

dict_keys(['rsdo', 'umo', 'vmo', 'mlotst', 'so', 'thetao', 'uo', 'vo', 'wfo', 'tos'])

In [3]:
import concurrent.futures
from cmipper import utils, config

def test_func(arg1, arg2, arg3):
    print(arg1, arg2, arg3)

def execute_functions_in_threadpool(args):
    try:
        with concurrent.futures.ThreadPoolExecutor(max_workers=12) as executor:
            futures = [executor.submit(parallelised_download_and_process.download_cmip_variable_data, *arg) for arg in args]
            return futures
    except Exception as e:
        print(f"An error occurred: {e}")


def main():
    model_info_dict = utils.read_yaml(config.model_info)
    download_config_dict = utils.read_yaml(config.download_config)
    limited_download_dict = utils.limit_model_info_dict(model_info_dict, download_config_dict)
    
    source_ids = ["EC-Earth3P-HR"]
    member_ids = limited_download_dict[source_ids[0]]["member_ids"]
    variable_ids = limited_download_dict[source_ids[0]]["variable_dict"].keys()

    try:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(execute_functions_in_threadpool, [(source_id, member_id, variable_id)]) 
                       for source_id in source_ids for member_id in member_ids for variable_id in variable_ids]

            # Wait for all futures to complete
            concurrent.futures.wait(futures)
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()


TIME CREATED: 2024-04-20 19:26:42

Processing data for EC-Earth3P-HR, r1i1p2f1, vmo



vmo: searching ESGF servers... 
TIME CREATED: 2024-04-20 19:26:42

Processing data for EC-Earth3P-HR, r1i1p2f1, thetao



thetao: TIME CREATED: 2024-04-20 19:26:42

Processing data for EC-Earth3P-HR, r1i1p2f1, umo



umo: searching ESGF servers... 
searching ESGF servers... 
TIME CREATED: 2024-04-20 19:26:42

Processing data for EC-Earth3P-HR, r1i1p2f1, uo



uo: searching ESGF servers... 
TIME CREATED: 2024-04-20 19:26:42

Processing data for EC-Earth3P-HR, r1i1p2f1, mlotst



mlotst: TIME CREATED: 2024-04-20 19:26:42

Processing data for EC-Earth3P-HR, r1i1p2f1, vo



vo: searching ESGF servers... 
searching ESGF servers... 
TIME CREATED: 2024-04-20 19:26:42

Processing data for EC-Earth3P-HR, r1i1p2f1, rsdo



rsdo: TIME CREATED: 2024-04-20 19:26:42

Processing data for EC-Earth3P-HR, r1i1p2f1, wfo



wfo: searching ESGF servers... 
searching ESGF servers... 
TIME CREATED: 2024-04-20 19:26:42

Pro

Error:curl error: Timeout was reached
curl error details: 


	extracting seafloor values...

	extracting seafloor values...

	extracting seafloor values...

	extracting seafloor values...



Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer


	0: saving vo_uncropped_sfl-20_tp_197301-197312.nc file: /maps-priv/maps/rt582/cmipper/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/newtest/og_grid/vo/vo_uncropped_sfl-20_tp_197301-197312.nc


...

Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer


	0: saving uo_uncropped_sfl-20_tp_201201-201212.nc file: /maps-priv/maps/rt582/cmipper/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/newtest/og_grid/uo/uo_uncropped_sfl-20_tp_201201-201212.nc


Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer


	0: saving thetao_uncropped_sfl-20_tp_198701-198712.nc file: /maps-priv/maps/rt582/cmipper/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/newtest/og_grid/thetao/thetao_uncropped_sfl-20_tp_198701-198712.nc


Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing layer
Error:curl error: Stream error in the HTTP/2 framing lay

# WIP: Attempt at parallelised logging

In [6]:
def main():
    model_info_dict = utils.read_yaml(config.model_info)
    download_config_dict = utils.read_yaml(config.download_config)

    source_ids = ["EC-Earth3P-HR"]
    member_ids = model_info_dict[source_ids[0]]["member_ids"]
    variable_ids = download_config_dict["variable_ids"]

    try:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = []
            for source_id in source_ids:
                for member_id in member_ids:
                    for variable_id in variable_ids:
                        log_fp = config.logging_dir / source_id / member_id / "_".join([variable_id, "download.log"])
                        print(log_fp)
                        if not log_fp.parent.exists():
                            log_fp.parent.mkdir(parents=True)
                        utils.redirect_stdout_stderr_to_file(log_fp)
                        # futures.extend(executor.submit(utils.execute_functions_in_threadpool, [(source_id, member_id, variable_id)]))
                        futures = [executor.submit(utils.execute_functions_in_threadpool, [(source_id, member_id, variable_id)]) 
                                for source_id in source_ids for member_id in member_ids for variable_id in variable_ids]

            # Wait for all futures to complete
            concurrent.futures.wait(futures)
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":

    main()